In [1]:
import os
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Rescaling
import matplotlib.pyplot as plt

In [2]:
# Define the directory path
data_dir = "./dataset/mtf_data_set"

In [3]:
img_width, img_height = 1000, 400
batch_size = 75

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    # labels='inferred',
    labels=None,
    label_mode='categorical',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    color_mode='rgb',
    shuffle=True,
    subset='training',
    seed=100,
    validation_split=0.2,
)

print(train_ds)

Found 75 files.
Using 60 files for training.
<_PrefetchDataset element_spec=TensorSpec(shape=(None, 400, 1000, 3), dtype=tf.float32, name=None)>


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    # labels='inferred',
    labels=None,
    label_mode='categorical',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    color_mode='rgb',
    shuffle=True,
    subset='validation',
    seed=100,
    validation_split=0.2
)

print(val_ds.element_spec)

Found 75 files.
Using 15 files for validation.
TensorSpec(shape=(None, 400, 1000, 3), dtype=tf.float32, name=None)


In [22]:
model = Sequential([
    Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    # Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    # Dense(128, activation='relu'),
    Dense(128, activation='relu')
])

In [23]:
print(model.summary())

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)              │ (None, 400, 1000, 3)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 398, 998, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_30 (Conv2D)                   │ (None, 396, 996, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 198, 498, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_31 (Conv2D)                   │ (None, 196, 496, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_32 (Conv2D)                   │ (None, 194, 494, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_24 (MaxPooling2D)      │ (None, 97, 247, 64)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_33 (Conv2D)                   │ (None, 95, 245, 128)        │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_34 (Conv2D)                   │ (None, 93, 243, 128)        │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_25 (MaxPooling2D)      │ (None, 46, 121, 128)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 712448)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │      91,193,472 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 91,480,480 (348.97 MB)

 Trainable params: 91,480,480 (348.97 MB)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_ds, epochs=10, validation_data=val_ds)



In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_accuracy)